# 语言模型开放题

# 数据集
本次开放题将与课程内容保持一致，将周杰伦从第一张专辑《Jay》到第十张专辑《跨时代》中的歌词作为训练语言模型的数据集，来训练一个语言模型，并在模型训练好后，用这个模型来创作新的歌词。

这里简介将此数据集转换成字符级循环神经网络及其变体所需要的输入格式的方法：

### 读取数据集

首先读取这个数据集，为了打印方便，我们把换行符替换成空格，并且打印前40个字符。

In [10]:
with open('/home/kesci/input/jaychou_lyrics4703/jaychou_lyrics.txt') as f:
        corpus_chars = f.read()
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')

corpus_chars[:40]

'想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在宇宙里 我每天每天每'

### 建立字符索引

我们将每个字符映射成一个从0开始的连续整数作为索引，以方便之后的数据处理。

为了得到索引，我们将数据集里所有不同字符取出来，然后将其逐一映射到索引来构造词典。

接着，打印`vocab_size`，即词典中不同字符的个数，又称词典大小。

之后，将训练数据集中每个字符转化为索引，并打印前20个字符及其对应的索引。

建立字符索引之后我们就可以对于数据集进行操作了。

In [11]:
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
print('vocab_size:', vocab_size)
corpus_indices = [char_to_idx[char] for char in corpus_chars]
sample = corpus_indices[:20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)

vocab_size: 2582
chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [903, 1776, 726, 2059, 1377, 1060, 1649, 903, 1776, 1779, 1149, 1553, 1621, 1843, 977, 1450, 1649, 903, 1776, 1779]


# 评价指标

在本次大作业中，我们使用困惑度（perplexity）来评价语言模型的好坏，困惑度是对交叉熵损失函数做指数运算后得到的值。

* 最佳情况下，模型总是把标签类别的概率预测为1，此时困惑度为1；
* 最坏情况下，模型总是把标签类别的概率预测为0，此时困惑度为正无穷；
* 基线情况下，模型总是预测所有类别的概率都相同，此时困惑度为类别个数。

显然，任何一个有效模型的困惑度必须小于类别个数。在语言模型中，困惑度必须小于词典大小`vocab_size`。

# 语言模型选取

以下进入本次大作业的正题——语言模型的分析与比较。

学员需要根据每个知识点介绍之后的问题进行理论分析、模型实现、结果汇总以及指标分析，尽可能完善地阐述实验收获以回答问题，并且以撰写报告的形式汇总自己的研究成果。

## 循环神经网络

在$n$元语法中，时间步$t$的词$w_t$基于前面所有词的条件概率只考虑了最近时间步的$n-1$个词。如果要考虑比$t-(n-1)$更早时间步的词对$w_t$的可能影响，模型参数的数量将随之呈指数级增长。

循环神经网络（recurrent neural network，RNN）并非刚性地记忆所有固定长度的序列，而是通过隐藏状态来存储之前时间步的信息。具体来说，在循环神经网络中，时间步$t$的隐藏变量的计算由当前时间步的输入和上一时间步的隐藏变量共同决定：

$\boldsymbol{H}_t = \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hh}  + \boldsymbol{b}_h).$

如下图所示，隐藏变量能够捕捉截至当前时间步的序列的历史信息，就像是神经网络当前时间步的状态或记忆一样。

![Image Name](https://zh.gluon.ai/_images/rnn.svg)


### 问题一：
- 解释为什么循环神经网络可以表达某时间步的词基于文本序列中所有过去的词的条件概率？
- 调节循环神经网络的超参数以及深度，分析不同超参和深度下，训练时间、语言模型困惑度（perplexity）以及创作歌词的结果等相关指标的变化，并以表格的形式进行总结。

## 门控循环神经网络


门控循环神经网络（gated recurrent neural network）通过可以学习的门来控制信息的流动，可以更好地捕捉时间序列中时间步距离较大的依赖关系。


### 门控循环单元（GRU）





门控循环单元（gated recurrent unit，GRU）是一种常用的门控循环神经网络[1, 2]，它引入了重置门（reset gate）和更新门（update gate）的概念，并且计算候选隐藏状态，从而修改了循环神经网络中隐藏状态的计算方式。

* 重置门控制了上一时间步的隐藏状态如何流入当前时间步的候选隐藏状态，有助于捕捉时间序列里短期的依赖关系；
* 更新门可以控制隐藏状态应该如何被包含当前时间步信息的候选隐藏状态所更新，有助于捕捉时间序列里长期的依赖关系。

如下图所示，假设更新门在时间步$t'$到$t$（$t' < t$）之间一直近似1，那么，在时间步$t'$到$t$之间的输入信息几乎没有流入时间步$t$的隐藏状态$\boldsymbol{H}_t$。实际上，这可以看作是较早时刻的隐藏状态$\boldsymbol{H}_{t'-1}$一直通过时间保存并传递至当前时间步$t$，时间步$t$的候选隐藏状态$\tilde{\boldsymbol{H}}_t \in \mathbb{R}^{n \times h}$来辅助稍后的隐藏状态计算，使得模型更好地捕捉时间序列中时间步距离较大的依赖关系。


![Image Name](https://zh.gluon.ai/_images/gru_3.svg)



### 长短期记忆（LSTM）


长短期记忆（long short-term memory，LSTM）[3] 网络，比门控循环单元的结构稍微复杂一点，它在循环神经网络的基础上引入了3个门，即输入门（input gate）、遗忘门（forget gate）和输出门（output gate），以及与隐藏状态形状相同的记忆细胞记录额外的信息。

* 遗忘门控制上一时间步的记忆细胞$\boldsymbol{C}_{t-1}$中的信息是否传递到当前时间步
* 输入门则控制当前时间步的输入$\boldsymbol{X}_t$通过候选记忆细胞$\tilde{\boldsymbol{C}}_t$如何流入当前时间步的记忆细胞


如下图所示，LSTM可以通过元素值域在$[0, 1]$的输入门、遗忘门和输出门来控制隐藏状态中信息的流动。当前时间步记忆细胞$\boldsymbol{C}_t \in \mathbb{R}^{n \times h}$的计算组合了上一时间步记忆细胞和当前时间步候选记忆细胞的信息。有了记忆细胞以后，接下来LSTM可以通过输出门来控制从记忆细胞到隐藏状态$\boldsymbol{H}_t \in \mathbb{R}^{n \times h}$的信息的流动。


![Image Name](https://zh.gluon.ai/_images/lstm_3.svg)


### 问题二：

- 请比较循环神经网络与门控循环网络之间结构的联系以及区别，并且从理论的角度描述门控循环神经网络的出现是在尝试解决循环神经网络中存在的哪几方面不足的。
- 调节门控循环单元（GRU）以及长短期记忆（LSTM）的超参数以及深度，分析不同超参和深度下，训练时间、语言模型困惑度（perplexity）以及创作歌词的结果等相关指标的变化，并以表格的形式进行总结。

## 简单循环单元（SRU）


在保持整个模型的设计思路不发生改变的情况下，深度学习的瓶颈往往在于计算。考虑到上述循环神经网络无法进行并行运算，因此往往需要需要大量的训练时间。

简单循环单元（simple recurrent unit，SRU）[4]旨在提出和探索简单快速并更具解释性的循环神经网络，因此它在门控循环网络的结构上进行了改进。


如下图所示，SRU的模型结构去除了遗忘门（forget gate）以及重置门（reset gate）对于上一时刻隐藏状态的依赖，以便于计算机实现并行化处理。

![Image Name](https://i.imgur.com/ahILNr0.png)



### 问题三：

- 通过阅读论文，试着详细分析简单循环单元（SRU）还在哪些方面进行了计算优化，尝试解决循环神经网络无法并行训练的问题，从而大大提高了训练速度。
- 调节简单循环单元（SRU）的超参数以及深度，分析不同超参和深度下，训练时间、语言模型困惑度（perplexity）以及创作歌词的结果等相关指标的变化，并以表格的形式进行总结，并与之前的网络结构进行对比分析。

### 参考文献

[1] Cho, K., Van Merriënboer, B., Bahdanau, D., & Bengio, Y. (2014). On the properties of neural machine translation: Encoder-decoder approaches. arXiv preprint arXiv:1409.1259.

[2] Chung, J., Gulcehre, C., Cho, K., & Bengio, Y. (2014). Empirical evaluation of gated recurrent neural networks on sequence modeling. arXiv preprint arXiv:1412.3555.

[3] Hochreiter, S., & Schmidhuber, J. (1997). Long short-term memory. Neural computation, 9(8), 1735-1780.

[4] Lei, T., Zhang, Y., Wang, SI., Dai, H., & Artzi, Y. (2017). Simple recurrent units for highly parallelizable recurrence. arXiv preprint arXiv:1709.02755

## 项目报告
本次大作业的终审评估以项目报告作为重要依据，开放题报告的内容和排版要求请下载文件：


[termproject1.zip](https://boyuai.oss-cn-shanghai.aliyuncs.com/disk/YouthAI%E7%A7%8B%E5%AD%A3%E6%80%9D%E7%BB%B4%E7%8F%AD-%E4%B8%8A%E8%AF%BE%E8%A7%86%E9%A2%91/termproject1.zip)

需要注意的是，文件中：
- `termproject.pdf`提供了项目报告的内容格式要求
- `termproject_exp.pdf`提供了项目报告的内容排版样例


推荐使用`LaTeX`软件进行报告的撰写，相关`.tex`以及`.sty`源文件一并附于文件夹中。
